In [1]:
### Different approach - batch size learning 
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from client import Client
from fedavg import Fedavg
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import random
import utils

In [2]:
selected_model = Supported_modles.SGD_classifier

In [3]:
clients = utils.set_data()
f1_before = []
for client in clients:
    client.split_data()
    # client.prep_data()
    client.train_model(selected_model)
    f1_before.append(client.F1)

In [4]:
f1_before

[0.7339671564390665,
 0.7546604527296937,
 0.8212765957446808,
 0.8182716049382717,
 0.6948523479102793]

In [5]:
model = clients[0].model
clients = utils.set_data()
for client in clients:
    client.split_data()
    client.init_empty_model(selected_model,model)

In [6]:
fedavg = Fedavg("global")
lr = 0.05
fedavg.init_global_model(lr, selected_model, clients[0].model, 57)

In [7]:
number_of_rounds = 20
batch_size = 0.05
epochs = 20
# prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!
Starting new round!


In [8]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.7931682858038231, 0.7739511584220412, 0.7887143315165117, 0.7758284600389864, 0.7818538224586951]


In [9]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.05920112936475663
Difference 0.019290705692347476
Difference -0.0325622642281691
Difference -0.042443144899285246
Difference 0.08700147454841578
